In [1]:
# enable this to download the file if not already present

#!wget http://s3.amazonaws.com/open.source.geoscience/open_data/newzealand/Taranaiki_Basin/Keri_3D/Kerry3D.segy

In [2]:
import numpy as np
from numba import vectorize, cuda
from struct import unpack
from ibm2ieee import ibm2float32 as i2f
import segyio
from segyio import _segyio
import cupy as cp
%config Completer.use_jedi = False

In [8]:
import sys
sys.path.insert(0, '/home/greg/Projects/pydata-segy/')
from segytools import *

In [ ]:
class ReadBinHdr(object):

    ''' Read binary trace header for a trace '''

    def __init__(self, bh):
        if len(bh) != 400:
            print("Binary header should be 400 bytes long")
        else:
            self.jobid = unpack(">i", bh[0:4])[0]
            self.line = unpack(">i", bh[4:8])[0]
            self.reel = unpack(">i", bh[8:12])[0]
            self.numtrcens = unpack(">h", bh[12:14])[0]
            self.numauxtrcs = unpack(">h", bh[14:16])[0]
            self.sampint = unpack(">h", bh[16:18])[0]
            self.sampint2 = unpack(">h", bh[18:20])[0]
            self.samppertrc = unpack(">h", bh[20:22])[0]
            self.samppertrc2 = unpack(">h", bh[22:24])[0]
            self.datasampcode = unpack(">h", bh[24:26])[0]
            self.ensfold = unpack(">h", bh[26:28])[0]
            self.sortcode = unpack(">h", bh[28:30])[0]
            self.dis_units = unpack(">h", bh[54:56])[0]
            self.segyformat = unpack(">h", bh[300:302])[0] // 256
            self.lengthflag = unpack(">h", bh[302:304])[0]
            self.numexthdrs = unpack(">h", bh[304:306])[0]
    
class ReadTrcHdr(object):
    ''' Read Trace header '''
    def __init__(self, bh):
        if len(bh) != 240:
            print("Trace Header should be 240 bytes long")
        else:
            self.inline = unpack(">i", bh[220:224])[0]
            self.xline = unpack(">i", bh[20:24])[0]
            self.sou_x = unpack(">i", bh[72:76])[0]
            self.sou_y = unpack(">i", bh[76:80])[0]

In [ ]:
%%time
hdrs = []
trcs = []
with open("Kerry3D.segy","rb") as f:
    EBCDIC = f.read(3200)
    BIN = ReadBinHdr(f.read(400))
    samples_per_trace = BIN.samppertrc
    sample_interval = BIN.sampint
    idx=1
    print("Starting loop....")
    while True:
        tmp = f.read(240)
        if not tmp:
            print("End scanning .......")
            break
        hdr = ReadTrcHdr(tmp).__dict__
        trc = f.read(samples_per_trace*4)
#         arr = ibmtoieee(np.frombuffer(trc,dtype='>u4'))
        hdrs.append(hdr)
        trcs.append(trc)
        idx+=1
data = trcs[10000] #data to perform analysis on
data[:100]

In [ ]:
# the Cuda Kernel code that runs on every unsigned int in array
ibm32cupy = cp.RawKernel(r'''
extern "C" __global__
void ibm32cupy(const unsigned int* x1, float* y) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int x = x1[tid];
    if (x != 0){
        int sign = ((x1[tid] >> 31) & 0x01) * (-2) + 1;
        int exponent = (x1[tid] >> 24) & 0x7F;
        int tmp = 4 * (exponent - 64);
        double p;
        if (tmp < 0) {
            int po2 = 1 << (abs(tmp));
            p = (double)(1.0/po2);
        }
        else{
            p = 1 << tmp;
        }
        int mantissa = x1[tid] & 0x00ffffff;
        float frac = ((float)mantissa / 0x1000000);
        y[tid] = sign * frac * p;
    }
    else{
        y[tid] = 0.0;
    }    
}
''', 'ibm32cupy')
# vectorized version of regular python function
@vectorize(['float32(uint32)'])
def ibmpy_vec(data):
    if data == 0:
        return 0.0
    sign = data >> 31 & 0x01
    exponent = data >> 24 & 0x7f
    mantissa = (data & 0x00ffffff) / float(pow(2, 24))
    return (1 - 2 * sign) * mantissa * pow(16.0, exponent - 64)

#reguar python
def ibmpython(data):
    if data == 0:
        return 0.0
    sign = data >> 31 & 0x01
    exponent = data >> 24 & 0x7f
    mantissa = (data & 0x00ffffff) / float(pow(2, 24))
    return (1 - 2 * sign) * mantissa * pow(16.0, exponent - 64)

In [ ]:
# intuitive first attempt of converting. apply a function to every element of an array in loop
%timeit np.array([ibmpython(x) for x in np.frombuffer(data,dtype=">u4")],dtype=np.float32)

In [ ]:
#vectorized version of previous attempt. 
%timeit np.vectorize(ibmpython)(np.frombuffer(data,dtype='>u4'))

In [ ]:
# uses the current industry method of segyio module
offset = 3600 + 10001 * (240 + (samples_per_trace * 4))
d = np.memmap('Kerry3D.segy', offset = offset, dtype = np.uint32)
%timeit segyio.tools.native(d[240:240+samples_per_trace])

In [ ]:
#uses pypi module ibm2ieee which is cython under the hood. https://pypi.org/project/ibm2ieee/
%timeit i2f(np.frombuffer(data,dtype=">u4"))

In [ ]:
# uses a numba vectorized version of the python function. effectively precompiles to machine code.
%timeit ibmpy_vec(np.frombuffer(data,dtype='>u4'))

In [ ]:
# uses cupy with a raw kernel wriiten in cuda. memory output must be preallocated
x1 = cp.array(np.frombuffer(data,dtype='>u4'),dtype=cp.uint32)
arr = cp.zeros(x1.size, dtype=cp.float32)
%timeit ibm32cupy((x1.size,), (1,), (x1, arr))

In [ ]:
ans = np.array([ibmpython(x) for x in np.frombuffer(data,dtype=">u4")],dtype=np.float32)

In [ ]:
np.sum((ans - np.vectorize(ibmpython)(np.frombuffer(data,dtype='>u4')))**2)

In [ ]:
np.sum((ans - segyio.tools.native(d[240:240+samples_per_trace],format=1))**2)

In [ ]:
np.sum((ans - i2f(np.frombuffer(data,dtype=">u4")))**2)

In [ ]:
np.sum((ans - ibmpy_vec(np.frombuffer(data,dtype='>u4')))**2)

In [ ]:
cp.sum((cp.array(ans,dtype=cp.float32) - arr)**2)

In [ ]:
d[239:240+samples_per_trace][:120]

In [ ]:
np.frombuffer(data,dtype=">u4")[:100]

In [4]:
i2f(np.array([1605997759,]))

TypeError: ufunc 'ibm2float32' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''